###여러 독립변수와 종속변수 사용

In [1]:
import tensorflow as tf
tf.__version__

'2.9.2'

###Raw Data Loading

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./bmi.csv', skiprows=3)
# df
# df.describe()
# df.info()
df.keys()

Index(['label', 'height', 'weight'], dtype='object')

In [4]:
train_df = df[['height', 'weight', 'label',]]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   height  20000 non-null  int64
 1   weight  20000 non-null  int64
 2   label   20000 non-null  int64
dtypes: int64(3)
memory usage: 468.9 KB


###Data PreProcessing

#####각 columns 분리

In [5]:
feature_set = train_df[['height', 'weight',]] # Dont Need 2차원 Numpy 변환
feature_set.head()

,height,weight
0,188,71
1,161,68
2,178,52
3,136,63
4,145,52


In [6]:
target_set = train_df['label'].values.reshape(-1,1) # 2차원 Numpy 변환
target_set.shape

(20000, 1)

####data Split
- train:test(7:3)

In [7]:
from sklearn.model_selection import train_test_split

feature_train, feature_test, target_train, target_test = train_test_split(feature_set, target_set, test_size=0.3)

####Missing Value(결측치)

####이상치 처리 : 생략

####정규화(Normalization)

#####Min-Max Scaling

In [8]:
from sklearn.preprocessing import MinMaxScaler

#####feature data(train and test)

######Train data

In [9]:
feature_train_minmaxscaler = MinMaxScaler()
feature_train_minmaxscaler.fit(feature_train)
feature_train_minmaxscaler.scale_, feature_train_minmaxscaler.min_

(array([0.0125    , 0.02222222]), array([-1.5       , -0.77777778]))

In [10]:
feature_train_normalization = feature_train_minmaxscaler.transform(feature_train)
feature_train_normalization.shape

(14000, 2)

######Test data

In [11]:
feature_test_minmaxscaler = MinMaxScaler()
feature_test_minmaxscaler.fit(feature_test)
feature_test_minmaxscaler.scale_, feature_test_minmaxscaler.min_

(array([0.0125    , 0.02222222]), array([-1.5       , -0.77777778]))

In [12]:
feature_test_normalization = feature_test_minmaxscaler.transform(feature_test)
feature_test_normalization.shape

(6000, 2)

#####target data(train and test)

##### rename

In [13]:
feature_train = feature_train_normalization.copy()
feature_test = feature_test_normalization.copy()
#target_train = target_train_normalization.copy()
#target_test = target_test_normalization.copy()

###Model 구현

In [14]:
import tensorflow as tf

#### input layer

In [15]:
input = tf.keras.layers.Input(shape=(2,))   # feature column count

####hidden layers

In [16]:
hiddens = tf.keras.layers.Flatten()(input)
hiddens = tf.keras.layers.Dense(125, activation='relu')(hiddens)

#### output layer

In [17]:
output = tf.keras.layers.Dense(units=3, activation='softmax')(hiddens)  # output classes count

#### Model 

In [18]:
model = tf.keras.Model(inputs=input, outputs=output)

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense (Dense)               (None, 125)               375       
                                                                 
 dense_1 (Dense)             (None, 3)                 378       
                                                                 
Total params: 753
Trainable params: 753
Non-trainable params: 0
_________________________________________________________________


###Model 설정

####Optimizer 선택

In [20]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)

####Loss function 선택

In [21]:
#loss = tf.keras.losses.MeanSquaredError()
#loss = tf.keras.losses.BinaryCrossentropy()
loss = tf.keras.losses.SparseCategoricalCrossentropy()

####Model 평가치 선택
- 데이터 형태 따라 평가치를 선택
- accuracy, precision, recall, fi score, AUC curve, logloss

In [22]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

###Model 학습

In [23]:
epoch = 50
history = model.fit(x=feature_train, y=target_train, validation_split=0.3, epochs=epoch, shuffle=True, verbose=0)

###Model Evaluate

In [24]:
result_evaluation = model.evaluate(feature_test, target_test)
result_evaluation

188/188 [==============================] - 0s 998us/step - loss: 0.1121 - accuracy: 0.9823


[0.11208099126815796, 0.9823333621025085]